In [2]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# Dataset path
train_path = 'HAM10000/train_dir'
test_path = 'HAM10000/test_dir'

In [4]:
# Using VGG16 from given model of tenserflow keras
Image_size = [224, 224, 3]
VGG_16 = VGG16(input_shape=Image_size, weights='imagenet', include_top=False,pooling='max')   

In [5]:
for layer in VGG_16.layers:
    layer.trainable = False

In [6]:
folders = glob('HAM10000/train_dir/*')
print(len(folders))

7


In [7]:
x = Flatten()(VGG_16.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=VGG_16.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [30]:
from keras import optimizers


adam = tf.keras.optimizers.Adam(learning_rate=0.01,epsilon=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [8]:
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [9]:
test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [10]:
train_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                  shuffle=True)

Found 6372 images belonging to 7 classes.


In [11]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                            shuffle = False)

Found 1525 images belonging to 7 classes.


In [ ]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint = ModelCheckpoint(filepath='vgg16_trans.hdf5', 
                               verbose=2, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model_history=model.fit(
  train_set, 
  validation_data=test_set,
  epochs=100,
  steps_per_epoch=350,
  validation_steps=50,
  callbacks=callbacks ,verbose=2)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100

Epoch 1: val_loss improved from inf to 0.80625, saving model to vgg16_trans.hdf5
350/350 - 72s - loss: 1.0594 - accuracy: 0.5898 - val_loss: 0.8063 - val_accuracy: 0.7163 - 72s/epoch - 205ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.80625 to 0.78029, saving model to vgg16_trans.hdf5
350/350 - 70s - loss: 1.1204 - accuracy: 0.5689 - val_loss: 0.7803 - val_accuracy: 0.7462 - 70s/epoch - 201ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 0.78029
350/350 - 70s - loss: 1.0966 - accuracy: 0.5798 - val_loss: 0.8739 - val_accuracy: 0.7000 - 70s/epoch - 200ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.78029
350/350 - 70s - loss: 1.0934 - accuracy: 0.5782 - val_loss: 0.8612 - val_accuracy: 0.7212 - 70s/epoch - 201ms/step
Epoch 5/100

Epoch 5: val_loss did not improve from 0.78029
350/350 - 70s - loss: 1.1337 - accuracy: 0.5752 - val_loss: 0.8944 - val_accuracy: 0.7362 - 70s/epoch - 201ms/step
Epoch 6/100

Epoch 6: val_loss did not improve from 0

350/350 - 71s - loss: 1.0772 - accuracy: 0.5830 - val_loss: 0.7870 - val_accuracy: 0.7437 - 71s/epoch - 203ms/step
Epoch 47/100

Epoch 47: val_loss did not improve from 0.74400
350/350 - 71s - loss: 1.0792 - accuracy: 0.5812 - val_loss: 0.8054 - val_accuracy: 0.7275 - 71s/epoch - 202ms/step
Epoch 48/100

Epoch 48: val_loss did not improve from 0.74400
350/350 - 71s - loss: 1.0840 - accuracy: 0.5870 - val_loss: 0.8216 - val_accuracy: 0.7038 - 71s/epoch - 202ms/step
Epoch 49/100

Epoch 49: val_loss did not improve from 0.74400
350/350 - 72s - loss: 1.0707 - accuracy: 0.5906 - val_loss: 0.8205 - val_accuracy: 0.7063 - 72s/epoch - 205ms/step
Epoch 50/100

Epoch 50: val_loss did not improve from 0.74400
350/350 - 71s - loss: 1.0964 - accuracy: 0.5934 - val_loss: 0.7529 - val_accuracy: 0.7575 - 71s/epoch - 203ms/step
Epoch 51/100

Epoch 51: val_loss did not improve from 0.74400
350/350 - 71s - loss: 1.0843 - accuracy: 0.5891 - val_loss: 0.8119 - val_accuracy: 0.7287 - 71s/epoch - 203ms/step


In [12]:
from tensorflow.keras import models
model.load_weights("vgg16_trans.hdf5")

In [14]:
predictions = model.predict(test_set, steps=1525/16, verbose=0)

In [16]:
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
#geting predictions on test dataset
y_pred = np.argmax(predictions, axis=1)

targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
#getting the true labels per image 
y_true = test_set.classes

#getting the predicted labels per image 
y_prob=predictions
from tensorflow.keras.utils import to_categorical
y_test = to_categorical(y_true)

# Creating classification report 
report = classification_report(y_true, y_pred, target_names=targetnames)

print("\nClassification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

       akiec       0.59      0.25      0.35        40
         bcc       0.45      0.71      0.55        48
         bkl       0.36      0.51      0.42       124
          df       0.00      0.00      0.00        11
         mel       0.44      0.22      0.29        63
          nv       0.92      0.91      0.91      1222
        vasc       0.80      0.47      0.59        17

    accuracy                           0.82      1525
   macro avg       0.51      0.44      0.45      1525
weighted avg       0.82      0.82      0.81      1525



In [17]:
from sklearn.metrics import roc_auc_score
print("Precision: "+ str(precision_score(y_true, y_pred, average='weighted')))
print("Recall: "+ str(recall_score(y_true, y_pred, average='weighted')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred)))
print("weighted Roc score: " + str(roc_auc_score(y_true,y_prob,multi_class='ovr',average='weighted')))

Precision: 0.8197933176361285
Recall: 0.8150819672131148
Accuracy: 0.8150819672131148
weighted Roc score: 0.8871492491641936


In [18]:
fpr = {}
tpr = {}
roc_auc = {}
for i in range(7):
    r = roc_auc_score(y_test[:, i], y_prob[:, i])
    print("The ROC AUC score of "+targetnames[i]+" is: "+str(r))

The ROC AUC score of akiec is: 0.9082491582491583
The ROC AUC score of bcc is: 0.9498561272850372
The ROC AUC score of bkl is: 0.848437751836246
The ROC AUC score of df is: 0.9461390656899242
The ROC AUC score of mel is: 0.9198097843788678
The ROC AUC score of nv is: 0.8852095520517682
The ROC AUC score of vasc is: 0.9230379154314245
